In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yufengdev/bbc-fulltext-and-category")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'bbc-fulltext-and-category' dataset.
Path to dataset files: /kaggle/input/bbc-fulltext-and-category


In [3]:
import pandas as pd
import numpy as np
import random
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, accuracy_score
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,Bidirectional,Dropout,Embedding,GlobalAveragePooling1D,BatchNormalization,LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam , Adamax
import re
import spacy
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("/kaggle/input/bbc-fulltext-and-category/bbc-text.csv")

In [5]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [10]:
df.isna().sum()

,0
category,0
text,0


In [12]:
df.duplicated().sum()

np.int64(99)

In [13]:
df.drop_duplicates(inplace = True )

In [14]:
df.duplicated().sum()

np.int64(0)

In [15]:
df["category"].value_counts()

,count
category,
sport,504
business,503
politics,403
entertainment,369
tech,347


In [16]:
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

In [17]:
def preprocessing(text, keep_stopwords=False):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_space or token.is_punct:
            continue
        if not keep_stopwords and token.is_stop:
            continue
        lemma = token.lemma_.strip()
        if lemma and lemma != "-PRON-" and len(lemma) > 2:  # Add minimum length
            tokens.append(lemma)
    return " ".join(tokens)

df['clean_text'] = df['text'].apply(lambda x: preprocessing(x, keep_stopwords=False))

In [18]:
df

,category,text,clean_text
0,tech,tv future in the hands of viewers with home th...,future hand viewer home theatre system plasma ...
1,business,worldcom boss left books alone former worldc...,worldcom boss leave book worldcom boss bernie ...
2,sport,tigers wary of farrell gamble leicester say ...,tiger wary farrell gamble leicester rush make ...
3,sport,yeading face newcastle in fa cup premiership s...,yeade face newcastle cup premiership newcastle...
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean raid box office ocean crime caper sequel...
...,...,...,...
2220,business,cars pull down us retail figures us retail sal...,car pull retail figure retail sale fall januar...
2221,politics,kilroy unveils immigration policy ex-chatshow ...,kilroy unveil immigration policy chatshow host...
2222,entertainment,rem announce new glasgow concert us band rem h...,rem announce new glasgow concert band rem anno...
2223,politics,how political squabbles snowball it s become c...,political squabble snowball commonplace argue ...


In [19]:
X = df['clean_text'].values
y = df['category'].values

In [20]:
X_train_texts, X_test_texts, y_train_lbls, y_test_lbls = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y)

In [21]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_texts)

In [22]:
X_train_seq = tokenizer.texts_to_sequences(X_train_texts)
X_test_seq  = tokenizer.texts_to_sequences(X_test_texts)

In [23]:
sequence_lengths = [len(seq) for seq in X_train_seq]
maxlen = int(np.percentile(sequence_lengths, 95))
print(f"Using maxlen: {maxlen}")

X_train = pad_sequences(X_train_seq, maxlen=maxlen, padding='post', truncating='post')
X_test = pad_sequences(X_test_seq, maxlen=maxlen, padding='post', truncating='post')

Using maxlen: 361


In [24]:
le = LabelEncoder()
y_train = le.fit_transform(y_train_lbls)
y_test  = le.transform(y_test_lbls)

In [25]:
num_classes = len(le.classes_)
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_cat  = tf.keras.utils.to_categorical(y_test, num_classes)

print("Classes:", le.classes_)

Classes: ['business' 'entertainment' 'politics' 'sport' 'tech']


In [26]:
maxlen=400

In [27]:
rnn = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=maxlen),

    Bidirectional(SimpleRNN(256, return_sequences=True)),
    Dropout(0.2),

    Bidirectional(SimpleRNN(128, return_sequences=True)),
    Dropout(0.2),

    Bidirectional(SimpleRNN(64)),
    Dropout(0.2),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(5, activation='softmax')
])


rnn.compile(optimizer=Adamax(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
rnn.build(input_shape=(None, 400))
rnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 400, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 400, 512)       │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 400, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 400, 256)       │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 400, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,707,909 (6.52 MB)

 Trainable params: 1,707,653 (6.51 MB)

 Non-trainable params: 256 (1.00 KB)

In [29]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001
    ),

    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-6,
        verbose=1
    )
]

In [30]:
history_rnn=rnn.fit(X_train, y_train_cat,validation_split=0.2,epochs=50,batch_size=16,callbacks=callbacks,verbose=1)

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 31s 194ms/step - accuracy: 0.2147 - loss: 1.9516 - val_accuracy: 0.1941 - val_loss: 1.6880 - learning_rate: 0.0010
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step - accuracy: 0.2025 - loss: 1.8385 - val_accuracy: 0.2294 - val_loss: 1.7234 - learning_rate: 0.0010
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.2168 - loss: 1.8170 - val_accuracy: 0.2176 - val_loss: 1.6957 - learning_rate: 0.0010
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 152ms/step - accuracy: 0.2122 - loss: 1.7981 - val_accuracy: 0.1912 - val_loss: 1.6937 - learning_rate: 0.0010
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.2196 - loss: 1.7496 - val_accuracy: 0.2176 - val_loss: 1.7574 - learning_rate: 0.0010
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 152ms/step - accuracy: 0.2183 - loss: 1.7236 - val_accuracy: 0.2176 - val_loss: 1.6884 - learning_rate: 0.0010
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 152ms/step - accuracy: 0.2206 - loss: 1.

In [31]:
lstm = Sequential([
    Embedding(input_dim=20000, output_dim=200, input_length=maxlen),

    Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),

    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(5, activation='softmax')
])

lstm.compile(optimizer=Adam(learning_rate=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])
lstm.build(input_shape=(None, 400))

lstm.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 400, 200)       │     4,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 400, 256)       │       336,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,526,853 (17.27 MB)

 Trainable params: 4,526,597 (17.27 MB)

 Non-trainable params: 256 (1.00 KB)

In [32]:
history_lstm = lstm.fit(
    X_train, y_train_cat,
    batch_size=64,
    epochs=20,
    validation_data=(X_test, y_test_cat),
    callbacks=callbacks,
    verbose=1
)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 139s 5s/step - accuracy: 0.2245 - loss: 1.6934 - val_accuracy: 0.3005 - val_loss: 1.6025 - learning_rate: 5.0000e-04
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.4113 - loss: 1.4058 - val_accuracy: 0.6009 - val_loss: 1.4912 - learning_rate: 5.0000e-04
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.7340 - loss: 0.7278 - val_accuracy: 0.7488 - val_loss: 1.3214 - learning_rate: 5.0000e-04
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.9315 - loss: 0.2736 - val_accuracy: 0.8944 - val_loss: 1.0793 - learning_rate: 5.0000e-04
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.9766 - loss: 0.1080 - val_accuracy: 0.9085 - val_loss: 0.8847 - learning_rate: 5.0000e-04
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 119s 4s/step - accuracy: 0.9935 - loss: 0.0470 - val_accuracy: 0.9249 - val_loss: 0.7214 - learning_rate: 5.0000e-04
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.9938 

In [33]:
def eval_model(model, X, y_cat, y_true_int, label_names=None):
    loss, acc = model.evaluate(X, y_cat, verbose=0)
    probs = model.predict(X, verbose=0)
    preds = np.argmax(probs, axis=1)
    acc_score = accuracy_score(y_true_int, preds)
    precision, recall, f1, support = precision_recall_fscore_support(y_true_int, preds, average=None)
    macro_f1 = f1_score(y_true_int, preds, average='macro')
    weighted_f1 = f1_score(y_true_int, preds, average='weighted')
    report = classification_report(y_true_int, preds, target_names=label_names, digits=4)
    cm = confusion_matrix(y_true_int, preds)
    return {
        'loss': loss,
        'acc_metric_from_eval': acc,
        'accuracy': acc_score,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1,
        'precision_per_class': precision,
        'recall_per_class': recall,
        'f1_per_class': f1,
        'support': support,
        'report_text': report,
        'confusion_matrix': cm,
        'preds': preds,
        'probs': probs
    }

In [34]:
def plot_confusion(cm, classes, title="Confusion Matrix", normalize=False, figsize=(6,5)):
    plt.figure(figsize=figsize)
    cm_plot = cm.astype('float')
    if normalize:
        cm_sum = cm.sum(axis=1)[:, np.newaxis] + 1e-8
        cm_plot = cm_plot / cm_sum
    sns.heatmap(cm_plot, annot=True, fmt='.2f' if normalize else 'd', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.title(title)
    plt.show()

In [35]:
def plot_training_histories(histories, names=['model1','model2'], metric='accuracy'):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    for h, n in zip(histories, names):
        if metric in h.history:
            plt.plot(h.history[metric], label=f"{n} train")
        val_key = 'val_' + metric if not metric.startswith('val_') else metric
        if val_key in h.history:
            plt.plot(h.history[val_key], '--', label=f"{n} val")
    plt.title(metric)
    plt.legend()
    plt.subplot(1,2,2)
    for h, n in zip(histories, names):
        if 'loss' in h.history:
            plt.plot(h.history['loss'], label=f"{n} train_loss")
        if 'val_loss' in h.history:
            plt.plot(h.history['val_loss'], '--', label=f"{n} val_loss")
    plt.title('loss')
    plt.legend()
    plt.show()


In [36]:
res_lstm = eval_model(lstm, X_test, y_test_cat, y_test, label_names=getattr(le, 'classes_', None))
res_rnn  = eval_model(rnn, X_test, y_test_cat, y_test, label_names=getattr(le, 'classes_', None))

In [37]:
print("=== LSTM Test ===")
print(f"Loss (eval): {res_lstm['loss']:.4f} | Accuracy (eval): {res_lstm['acc_metric_from_eval']:.4f}")
print(f"Accuracy (sklearn): {res_lstm['accuracy']:.4f} | Macro F1: {res_lstm['macro_f1']:.4f} | Weighted F1: {res_lstm['weighted_f1']:.4f}")
print(res_lstm['report_text'])

=== LSTM Test ===
Loss (eval): 0.1853 | Accuracy (eval): 0.9460
Accuracy (sklearn): 0.9460 | Macro F1: 0.9444 | Weighted F1: 0.9458
               precision    recall  f1-score   support

     business     0.9143    0.9505    0.9320       101
entertainment     0.9444    0.9189    0.9315        74
     politics     0.9870    0.9383    0.9620        81
        sport     0.9439    1.0000    0.9712       101
         tech     0.9538    0.8986    0.9254        69

     accuracy                         0.9460       426
    macro avg     0.9487    0.9412    0.9444       426
 weighted avg     0.9468    0.9460    0.9458       426



In [38]:
print("\n\n=== RNN Test ===")
print(f"Loss (eval): {res_rnn['loss']:.4f} | Accuracy (eval): {res_rnn['acc_metric_from_eval']:.4f}")
print(f"Accuracy (sklearn): {res_rnn['accuracy']:.4f} | Macro F1: {res_rnn['macro_f1']:.4f} | Weighted F1: {res_rnn['weighted_f1']:.4f}")
print(res_rnn['report_text'])



=== RNN Test ===
Loss (eval): 1.6776 | Accuracy (eval): 0.2441
Accuracy (sklearn): 0.2441 | Macro F1: 0.1207 | Weighted F1: 0.1419
               precision    recall  f1-score   support

     business     0.2615    0.1683    0.2048       101
entertainment     0.0000    0.0000    0.0000        74
     politics     0.3333    0.0123    0.0238        81
        sport     0.2402    0.8515    0.3747       101
         tech     0.0000    0.0000    0.0000        69

     accuracy                         0.2441       426
    macro avg     0.1670    0.2064    0.1207       426
 weighted avg     0.1823    0.2441    0.1419       426

